In [4]:
import datetime
import json

class Product:
    def __init__(self, name, category, price, quantity, expiration_date):
        self.name = name
        self.category = category
        self.price = price
        self.quantity = quantity
        self.expiration_date = datetime.datetime.strptime(expiration_date, '%Y-%m-%d')

    def is_expired(self):
        return self.expiration_date < datetime.datetime.now()

    def to_dict(self):
        return {
            'name': self.name,
            'category': self.category,
            'price': self.price,
            'quantity': self.quantity,
            'expiration_date': self.expiration_date.strftime('%Y-%m-%d')
        }

    @classmethod
    def from_dict(cls, data):
        return cls(data['name'], data['category'], data['price'], data['quantity'], data['expiration_date'])

class Inventory:
    def __init__(self):
        self.products = []

    def add_product_to_inventory(self, product):
        self.products.append(product)

    def remove_product_from_inventory(self, product_name):
        self.products = [p for p in self.products if p.name != product_name]

    def search_products(self, search_term):
        search_term = search_term.lower()
        return [p for p in self.products if search_term in p.name.lower() or search_term in p.category.lower()]

    def list_all_products(self):
        for p in self.products:
            print(f"{p.name} ({p.category}): {p.quantity} available, ${p.price} each, expires on {p.expiration_date.strftime('%Y-%m-%d')}")

    def categorize_products(self):
        categories = {}
        for p in self.products:
            if p.category not in categories:
                categories[p.category] = []
            categories[p.category].append(p.name)
        return categories

    def check_and_remove_expired_products(self):
        self.products = [p for p in self.products if not p.is_expired()]

    def save_to_file(self, filename):
        try:
            with open(filename, 'w') as file:
                json.dump([p.to_dict() for p in self.products], file)
        except IOError as e:
            print(f"Error saving to file: {e}")

    def load_from_file(self, filename):
        try:
            with open(filename, 'r') as file:
                self.products = [Product.from_dict(p) for p in json.load(file)]
        except (IOError, json.JSONDecodeError) as e:
            print(f"Error loading from file: {e}")

# Example usage:
if __name__ == "__main__":
    inventory = Inventory()

    apple = Product("Apple", "Fruit", 1.0, 50, "2024-06-30")
    banana = Product("Banana", "Fruit", 0.8, 30, "2024-06-25")
    milk = Product("Milk", "Dairy", 2.5, 20, "2024-06-28")

    inventory.add_product_to_inventory(apple)
    inventory.add_product_to_inventory(banana)
    inventory.add_product_to_inventory(milk)

    print("All products:")
    inventory.list_all_products()

    print("\nSearch results for 'apple':")
    for product in inventory.search_products("apple"):
        print(product.__dict__)

    print("\nCategorized products:")
    categorized = inventory.categorize_products()
    for category, products in categorized.items():
        print(f"{category}: {products}")

    inventory.check_and_remove_expired_products()
    print("\nProducts after removing expired ones:")
    inventory.list_all_products()

    inventory.save_to_file("inventory.json")
    print("\nSaved inventory to file.")

    new_inventory = Inventory()
    new_inventory.load_from_file("inventory.json")
    print("\nLoaded products from file:")
    new_inventory.list_all_products()

All products:
Apple (Fruit): 50 available, $1.0 each, expires on 2024-06-30
Banana (Fruit): 30 available, $0.8 each, expires on 2024-06-25
Milk (Dairy): 20 available, $2.5 each, expires on 2024-06-28

Search results for 'apple':
{'name': 'Apple', 'category': 'Fruit', 'price': 1.0, 'quantity': 50, 'expiration_date': datetime.datetime(2024, 6, 30, 0, 0)}

Categorized products:
Fruit: ['Apple', 'Banana']
Dairy: ['Milk']

Products after removing expired ones:
Apple (Fruit): 50 available, $1.0 each, expires on 2024-06-30
Banana (Fruit): 30 available, $0.8 each, expires on 2024-06-25
Milk (Dairy): 20 available, $2.5 each, expires on 2024-06-28

Saved inventory to file.

Loaded products from file:
Apple (Fruit): 50 available, $1.0 each, expires on 2024-06-30
Banana (Fruit): 30 available, $0.8 each, expires on 2024-06-25
Milk (Dairy): 20 available, $2.5 each, expires on 2024-06-28
